## Theory  
### Step 1: Define the Optimal Quantization Points for a Normal Distribution

The goal is to create a k-bit quantization scheme that best fits a standard normal 
distribution $N(0,1)$.

- We divide the normal distribution into $2^{k} + 1$ quantization bins.  
- The quantization points $q_{i}$ are computed using the quantile function 
$Q_{X}(.)$ of $N(0,1)$.  
- The quantization points are set as the midpoints between adjacent quantiles:  
$$ q_{i} = \frac{1}{2} * (Q_{X}(\frac{i}{2^{k} + 1}) + Q_{X}(\frac{i+1}{2^{k} + 1})) $$

where:  
- $Q_{X}(p)$ is the inverse cumulative distribution function (CDF) of $N(0,1)$.  
- $i$ ranges from $1 to 2^{k}$  

### Step 2: Normalize to the Range \[−1, 1\]  

After computing the quantization points for $N(0,1)$, we scale them to fit within the 
fixed range $[−1,1]$. This ensures compatibility with standard quantization methods.  

$$q^{'}_{i} = \frac{q_{i}}{max(\|q_{i}\|)}$$  

where $max(\|q_{i}\|)$ ensures that the values fit within $[−1,1]$.


In [ ]:
# From research paper
# NormalFloat 4-bit data type
# The exact values of the NF4 data type are as follows:

nf4_quantization_levels = [
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
]

### My Implementations

In [127]:
import torch

### 1. Naive implementation

In [129]:
# Naive implementation
k = 4
n_quantiles = 2**k + 1
normal_dist = torch.distributions.Normal(0, 1)

for i in range(1, n_quantiles):
    mid_points = 0.5 * (
        normal_dist.icdf(torch.tensor(i / n_quantiles))
        + normal_dist.icdf(torch.tensor((i + 1) / n_quantiles))
    )
    print(f"i: {i}, n_quantiles {n_quantiles}, mid_points: {mid_points}")

i: 1, n_quantiles 17, mid_points: -1.3757789134979248
i: 2, n_quantiles 17, mid_points: -1.0578655004501343
i: 3, n_quantiles 17, mid_points: -0.8252109289169312
i: 4, n_quantiles 17, mid_points: -0.6314586997032166
i: 5, n_quantiles 17, mid_points: -0.4593935012817383
i: 6, n_quantiles 17, mid_points: -0.30019986629486084
i: 7, n_quantiles 17, mid_points: -0.14839954674243927
i: 8, n_quantiles 17, mid_points: 3.725290298461914e-08
i: 9, n_quantiles 17, mid_points: 0.14839962124824524
i: 10, n_quantiles 17, mid_points: 0.3001999258995056
i: 11, n_quantiles 17, mid_points: 0.45939356088638306
i: 12, n_quantiles 17, mid_points: 0.6314587593078613
i: 13, n_quantiles 17, mid_points: 0.8252109289169312
i: 14, n_quantiles 17, mid_points: 1.0578655004501343
i: 15, n_quantiles 17, mid_points: 1.3757789134979248
i: 16, n_quantiles 17, mid_points: inf


Note:  
- The problem here is the infinite values, and no representation for zero

### 2. Handle the infinite values

In [ ]:
# Implement using linspace
## Define k-bit quantization
k = 4
n_quantiles = 2**k + 1  # 17 quantiles for NF4

## Standard normal distribution
normal_dist = torch.distributions.Normal(0, 1)

## Safe probability values avoiding 0 and 1
quantile_positions = torch.linspace(
    1 / (n_quantiles + 1), n_quantiles / (n_quantiles + 1), n_quantiles
)

## Compute mid-quantiles (symmetric NF4)
quantiles = 0.5 * (
    normal_dist.icdf(quantile_positions[:-1]) + normal_dist.icdf(quantile_positions[1:])
)
print(f"mid_quantiles: \n{quantiles}")

## Normalize
quantiles = quantiles.sort().values
print(f"sorted quantiles: \n{quantiles}")
quantiles /= quantiles.max()
print(f"Normalized quantiles: \n{quantiles}")


mid_quantiles: 
tensor([-1.4069, -1.0940, -0.8661, -0.6771, -0.5101, -0.3565, -0.2110, -0.0699,
         0.0699,  0.2110,  0.3565,  0.5101,  0.6771,  0.8661,  1.0940,  1.4069])
sorted q_values: 
tensor([-1.4069, -1.0940, -0.8661, -0.6771, -0.5101, -0.3565, -0.2110, -0.0699,
         0.0699,  0.2110,  0.3565,  0.5101,  0.6771,  0.8661,  1.0940,  1.4069])
Normalized q_values: 
tensor([-1.0000, -0.7776, -0.6156, -0.4812, -0.3626, -0.2534, -0.1499, -0.0497,
         0.0497,  0.1499,  0.2534,  0.3626,  0.4812,  0.6156,  0.7776,  1.0000])


Note:  
- The problem here is no representation for zero

### 3. Implementation from research paper

implementation from github modified to get NF4 quantization levels

Reference:  
- https://github.com/bitsandbytes-foundation/bitsandbytes/blob/1f2ca43ae5f3b453ff5fed73a17c661dc4fbbcb3/bitsandbytes/functional.py#L319

In [ ]:
# github implementation of the NF4
offset = 0.9677083
# one more positive value, this is an asymmetric type
normal_dist = torch.distributions.Normal(0, 1)
v1 = normal_dist.icdf(torch.linspace(offset, 0.5, 9)[:-1]).tolist()
print(f"v1 - {v1}")
# v2 = [0] * (256 - 15)  ## we have 15 non-zero values in this data type
v2 = [0]
print(f"v2 - {v2}")
v3 = (-normal_dist.icdf(torch.linspace(offset, 0.5, 8)[:-1])).tolist()
print(f"v3 -{v3}")

v = v1 + v2 + v3

values = torch.Tensor(v)
values = values.sort().values
print(f"sorted values: \n{values}")
values /= values.max()
print(f"normalized values: \n{values}")

v1 - [2.326348304748535, 1.4665447473526, 1.1146509647369385, 0.8641599416732788, 0.6588377356529236, 0.47821110486984253, 0.3120533227920532, 0.15413910150527954]
v2 - [0]
v3 -[-2.326348304748535, -1.40507173538208, -1.0364335775375366, -0.7721933126449585, -0.5533846616744995, -0.3584587275981903, -0.176374152302742]
sorted values: 
tensor([-2.3263, -1.4051, -1.0364, -0.7722, -0.5534, -0.3585, -0.1764,  0.0000,
         0.1541,  0.3121,  0.4782,  0.6588,  0.8642,  1.1147,  1.4665,  2.3263])
normalized values: 
tensor([-1.0000, -0.6040, -0.4455, -0.3319, -0.2379, -0.1541, -0.0758,  0.0000,
         0.0663,  0.1341,  0.2056,  0.2832,  0.3715,  0.4791,  0.6304,  1.0000])


### References:  

- https://chatgpt.com/share/67d23f45-b88c-800b-9dd7-dbe79008f7fe  
